In [ ]:
#import modin

In [ ]:
#modin.__version__

In [1]:
import pandas as pd
import pandas
import haystack
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.nodes import BM25Retriever
import tqdm
import numpy as np
import glob
import concurrent.futures
import multiprocessing
import requests

/opt/conda/envs/stackoverflow/lib/python3.8/site-packages/transformers/image_utils.py:222: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  def resize(self, image, size, resample=PIL.Image.BILINEAR, default_to_square=True, max_size=None):
/opt/conda/envs/stackoverflow/lib/python3.8/site-packages/redis/connection.py:67: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  hiredis_version = StrictVersion(hiredis.__version__)
/opt/conda/envs/stackoverflow/lib/python3.8/site-packages/redis/connection.py:69: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  hiredis_version >= StrictVersion('0.1.3')
/opt/conda/envs/stackoverflow/lib/python3.8/site-packages/redis/connection.py:71: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  hiredis_version >= StrictVersion('0.1.4')
/opt/conda/envs/s

In [2]:
import dask.dataframe as dd

In [3]:
#ray.init(log_to_driver=False)

In [4]:
!ls ../data

PostLinks.xml  df_raw		    stackoverflow.com-PostLinks.7z
Posts.xml      df_raw.parquet	    stackoverflow.com-Posts.7z
df_processed   nyc-restaurants.csv  start.sh


In [5]:
ELASTIC_HOST="np-database.c.np-training.internal"
ELASTIC_HOST="localhost"
ELASTIC_INDEX="stackoverflow"
ELASTIC_PORT=9200

In [6]:
requests.put(f"http://{ELASTIC_HOST}:{ELASTIC_PORT}/_template/index_defaults", json = 
    {
      "index_patterns": "*", 
      "settings": {
        "number_of_shards": 5
      }
    } 
).json()





{'acknowledged': True}

In [7]:
requests.delete(f"http://{ELASTIC_HOST}:{ELASTIC_PORT}/{ELASTIC_INDEX}").json()

{'error': {'root_cause': [{'type': 'index_not_found_exception',
    'reason': 'no such index [stackoverflow]',
    'resource.type': 'index_or_alias',
    'resource.id': 'stackoverflow',
    'index_uuid': '_na_',
    'index': 'stackoverflow'}],
  'type': 'index_not_found_exception',
  'reason': 'no such index [stackoverflow]',
  'resource.type': 'index_or_alias',
  'resource.id': 'stackoverflow',
  'index_uuid': '_na_',
  'index': 'stackoverflow'},
 'status': 404}

In [ ]:
# requests.put(f"http://{ELASTIC_HOST}:{ELASTIC_PORT}/{ELASTIC_INDEX}", json = 
#     {
#         "settings": {
#             "number_of_shards": 25,
#             "number_of_replicas": 2
#       }
#     } 
#     ) .json()





In [8]:
requests.get(f"http://{ELASTIC_HOST}:{ELASTIC_PORT}/_all/_settings").json()

{'nyc-restaurants': {'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}},
    'number_of_shards': '1',
    'provided_name': 'nyc-restaurants',
    'creation_date': '1666260357381',
    'number_of_replicas': '1',
    'uuid': '6B3HWt5QS9qgm0HhFlW0Ew',
    'version': {'created': '8040399'}}}}}

In [ ]:
!gsutil ls gs://np-training-tmp/stackoverflow/final/

In [ ]:
#!pip install gcsfs

In [9]:
files = ["gs://np-training-tmp/stackoverflow/final/posts.parquet"]

In [10]:
df = pd.read_parquet("gs://np-training-tmp/stackoverflow/final/posts.parquet")

In [11]:
df.head()

,Id,AcceptedAnswerId,Title,QuestionBody,Tags,ViewCount,AnswerCount,CommentCount,Score,CreationDate,AnswerId,AcceptedAnswerBody
0,70792825,NaN,PySide6 pixelates PNGs,When im trying to add a PNG to PySide6 GUI it loses in quality.\nHere is the...,"[python, qt, pyqt, pyside6]",40,0,11,0,2022-01-20 20:11:41.420,NaN,None
1,3595677,NaN,Are there any major performance differences between epoll and kqueue?,"My development machine is a MacBook (which of course has kqueue). However, ...","[python, asynchronous, tornado, epoll, kqueue]",2385,2,1,4,2010-08-29 16:57:50.367,NaN,None
2,71527749,NaN,Getting username from a multi-valued lookup field in Sharepoint List,"I am working with a Sharepoint List via REST API, managed to get all of the ...","[python, sharepoint, office365, office365api, sharepoint-list]",31,0,0,0,2022-03-18 13:26:32.360,NaN,None
3,60987111,NaN,debug python multiprocess in a multithreaded environment,I am trying to use python multiprocessing in an existing service - the exist...,"[python, multithreading, multiprocessing, pickle, deadlock]",132,0,14,0,2020-04-02 08:21:14.583,NaN,None
4,52413332,NaN,Iterating Through DataFrames,"I was having a hard time with the technical verbiage to word the question, s...","[python, pandas]",34,0,4,1,2018-09-19 19:55:20.473,NaN,None


In [ ]:
#df.shape

In [ ]:
#df.head(5)

In [42]:
df.iloc[0].to_dict()

{'Id': 70792825,
 'AcceptedAnswerId': nan,
 'Title': 'PySide6 pixelates PNGs',
 'QuestionBody': 'When im trying to add a PNG to PySide6 GUI it loses in quality.\nHere is the code:\nimport sys\nimport random\nfrom PySide6 import QtCore, QtWidgets, QtGui\nfrom PySide6.QtGui import QPalette, QColor\n\nBOARD_SIZE = 4\n\n\nclass MyWidget(QtWidgets.QWidget):\n    def __init__(self):\n        super().__init__()\n\n        self.resize(500, 300)\n        self.setWindowTitle("2048")\n\n        self.initUI()\n\n    def initUI(self):\n        grid = QtWidgets.QGridLayout()\n\n        # QtWidgets.QApplication.setGraphicsSystem(\'native\')\n\n        for i in range(BOARD_SIZE):\n            for j in range(BOARD_SIZE):\n                path = "./img/" + randomTile() + ".png"\n                img = QtGui.QImage(path)\n                pixmap = QtGui.QPixmap(img.scaledToWidth(150))\n                label = QtWidgets.QLabel()\n                label.setPixmap(pixmap)\n                grid.addWidget(label,

In [45]:
def convert_to_document_dict(row):
    d = row.to_dict()
    #d['content']=d['Title'] + "\n" + d['Body']
    
    d['content']=d['Title'] + "\n" #+ d['QuestionBody']
    d['id']= d['Id']
    d['Tags'] = d['Tags']
    
    del d['Id']
    del d['QuestionBody']
    del d['AcceptedAnswerBody']
    del d['AnswerId']
    del d['AcceptedAnswerId']

    
    # for k in list(d.keys()):
    #     if d[k] ==None  :
    #         del d[k]
    d_doc = haystack.schema.Document.from_dict(d)
    
    return d_doc


def fetch_results(query:str, retriever, top_k=10):
    candidate_documents = retriever.retrieve(
        query=query,
        top_k=10,
    #filters={"year": ["2015", "2016", "2017"]}
    )

    for doc in candidate_documents:
        print (doc.meta['Title'], doc.score, doc.id )
        
        
def index_df(df, document_store):
    docs = df.apply(convert_to_document_dict, axis=1).to_numpy().flatten().tolist()
    
    document_store.write_documents(docs, batch_size=1_000)
    
def index_individual_df(df, document_store):
    
    docs = df.apply(convert_to_document_dict, axis=1)\
            .apply(lambda x: document_store.write_documents ( [doc]))
    
    
    
def get_document_store():
    
    document_store = ElasticsearchDocumentStore(host=ELASTIC_HOST
                                            , index=ELASTIC_INDEX, port=ELASTIC_PORT)
    
    return document_store


def transform_batch(df: pd.DataFrame) -> pd.DataFrame:
    
    document_store = get_document_store()
    index_df(df,document_store)
    
    return df


def transform_batch2(df: pd.DataFrame) -> pd.DataFrame:
    
    document_store =  get_document_store()
    
    
    index_df(df,document_store)
    
    return len(df)



def transform_file_batch(path: str) -> pd.DataFrame:
    
    df = pandas.read_parquet(path)
    document_store = get_document_store()
    
    
    index_df(df,document_store)
    
    return len(df)



In [48]:
transform_batch2(df)

/opt/conda/envs/stackoverflow/lib/python3.8/site-packages/pandas/core/arrays/datetimelike.py:415: ResourceWarning: unclosed <socket.socket fd=73, family=AddressFamily.AF_INET6, type=SocketKind.SOCK_STREAM, proto=6, laddr=('::1', 45632, 0, 0), raddr=('::1', 9200, 0, 0)>
  converted = ints_to_pydatetime(
/opt/conda/envs/stackoverflow/lib/python3.8/site-packages/pandas/core/arrays/datetimelike.py:415: ResourceWarning: unclosed <socket.socket fd=65, family=AddressFamily.AF_INET6, type=SocketKind.SOCK_STREAM, proto=6, laddr=('::1', 60584, 0, 0), raddr=('::1', 9200, 0, 0)>
  converted = ints_to_pydatetime(
/opt/conda/envs/stackoverflow/lib/python3.8/site-packages/pandas/core/arrays/datetimelike.py:415: ResourceWarning: unclosed <socket.socket fd=67, family=AddressFamily.AF_INET6, type=SocketKind.SOCK_STREAM, proto=6, laddr=('::1', 45404, 0, 0), raddr=('::1', 9200, 0, 0)>
  converted = ints_to_pydatetime(
/opt/conda/envs/stackoverflow/lib/python3.8/site-packages/pandas/core/arrays/datetimelik

KeyboardInterrupt: 

In [49]:
ddf = dd.from_pandas(df, npartitions=5)

/opt/conda/envs/stackoverflow/lib/python3.8/site-packages/dask/base.py:1217: ResourceWarning: unclosed <socket.socket fd=72, family=AddressFamily.AF_INET6, type=SocketKind.SOCK_STREAM, proto=6, laddr=('::1', 36028, 0, 0), raddr=('::1', 9200, 0, 0)>
  data = hash_buffer_hex(pickle.dumps(x, pickle.HIGHEST_PROTOCOL))


In [ ]:
transform_batch2(df)

In [51]:
ddf.count().compute()

Id                    1960201
AcceptedAnswerId      1014936
Title                 1960201
QuestionBody          1960201
Tags                  1960201
ViewCount             1960201
AnswerCount           1960201
CommentCount          1960201
Score                 1960201
CreationDate          1960201
AnswerId              1014931
AcceptedAnswerBody    1014931
dtype: int64

In [52]:
ddf.map_partitions(transform_batch2, meta=('int64')).compute()

/opt/conda/envs/stackoverflow/lib/python3.8/site-packages/dask/dataframe/core.py:6597: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(


SystemExit: 15

/opt/conda/envs/stackoverflow/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f24bbd5e400>
Traceback (most recent call last):
  File "/opt/conda/envs/stackoverflow/lib/python3.8/site-packages/urllib3/connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "/opt/conda/envs/stackoverflow/lib/python3.8/site-packages/urllib3/connectionpool.py", line 449, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/opt/conda/envs/stackoverflow/lib/python3.8/site-packages/urllib3/connectionpool.py", line 444, in _make_request
    httplib_response = conn.getresponse()
  File "/opt/conda/envs/stackoverflow/lib/python3.8/http/client.py", line 1348, in getresponse
    response.begin()
  File "/opt/conda/envs/stackoverflow/lib/p

In [ ]:
cores = multiprocessing.cpu_count()-2

In [ ]:
# max_workers=10
with concurrent.futures.ProcessPoolExecutor(max_workers = cores) as executor:
    for file_path, num_records in zip(files, executor.map(transform_file_batch, files)):
        print(f"path:{file_path}; records:{num_records}")

In [ ]:
#curl http://localhost:9200/_cat/indices?v

In [ ]:
!ls

In [ ]:
df.head()

In [ ]:
pipeline = PandasQueryPipeline(df) 
pipeline.add_query(transform_batch, is_output=True)
result_dfs = pipeline.compute_batch() # Begin batch processing.

In [ ]:
!ls

In [ ]:
result_dfs

In [ ]:
ds.

In [ ]:
ds.limit(1000).map_batches(transform_batch)

In [ ]:
transformed_ds = ds.map_batches(transform_batch)


In [ ]:
transformed_ds.show()


In [ ]:
#convert_to_document_dict ( df.iloc[0] )

In [ ]:
#df.head(5).apply(convert_to_document_dict, axis=1)

In [ ]:
document_store.delete_documents()

In [ ]:
def apply_func(df):
    print ( len (df))



In [ ]:
df.apply(lambda s: apply_func(s.to_frame()), axis=1)

In [ ]:
@ray.remote
def index_df_with_remote(partition):
    df_subset = from_partitions( [ partitions ] ,axis=0)
      
    #docs = df_subset.apply(convert_to_document_dict, axis=1).to_numpy().flatten().tolist()
    
    #return  len (docs)
     
    return 5

In [ ]:
partitions = unwrap_partitions(df, axis=0, get_ip=True)


In [ ]:
futures = [index_df_with_remote.remote(partition) for partition in partitions]


In [ ]:
print(ray.get(futures)) # [0, 1, 4, 9]


In [ ]:
?document_store.write_documents

In [ ]:
index_df(df, document_store)

In [ ]:
# index_individual_df(df, document_store)

In [ ]:
!ls

In [ ]:
#partitions = unwrap_partitions(df.head(500))



In [ ]:

# partitions = unwrap_partitions(df, axis=0, get_ip=True)

# for i in tqdm.tqdm (range(len (partitions))):
    
#     df_subset = from_partitions(partitions[i:i+1],axis=0)
    
#     docs = df_subset.apply(convert_to_document_dict, axis=1).to_numpy().flatten().tolist()
    
#     document_store.write_documents(docs)

In [ ]:
document_store.write_documents(docs)

In [ ]:
document_store.get_document_count()

In [ ]:
fetch_results(query="encrypted URL", retriever=retriever)